In [ ]:
!pip install mimikit

In [ ]:
from mimikit.freqnet import FreqNet
from mimikit.data import Database
from mimikit.utils import audio
from mimikit import get_trainer, NeptuneConnector

In [ ]:
# neptune :

nep_con = NeptuneConnector(user="k-tonal",
                           setup=dict(db="data-and-base-notebooks/DAT-27",
                                      model="experiment-2-K3"))

db_name = "genoel-mix.h5"

# paths

path_to_db = "./" + db_name
path_to_model = "./experiment-2"


In [ ]:
# get your data

db = nep_con.download_database("db", db_name)
db.metadata

### Varying Parameters

we only try different layer-configurations together with the fastest possible params

In [ ]:
params = {
    # baselines :
    
    0: dict(n_layers=(1,)),
    1: dict(n_layers=(2,)),
    2: dict(n_layers=(3,)),
    3: dict(n_layers=(4,)),
    4: dict(n_layers=(5,)),
    
    # we change the compression-rate without changing the length of input_sequence to the net 
    # ==> thus we have more layers for as many input_time_steps :
    
    5: dict(n_layers=(1, 1, 1)),  ## == (2,)
    6: dict(n_layers=(1, 2, 2)),  ## == (3, )
    7: dict(n_layers=(1, 1, 1, 1, 1, 1, 1,)), ## == (3, )
    8: dict(n_layers=(1, 3, 3)),  ## == (4, )
    9: dict(n_layers=(1, 1, 2, 2, 3)),  ## == (4, )
    10: dict(n_layers=(3, 2, 2, 1, 1,)),  ## == (4, )
    11: dict(n_layers=(1, 4, 4)),  ## == (5, )
    12: dict(n_layers=(1, 1, 3, 3, 4)),  ## == (5, )
    13: dict(n_layers=(1, 3, 1, 3, 4)),  ## == (5, )
}

In [ ]:
# pick a parametrization :

PARAMS_INDEX = 9

# if you want ilocs (files in the db) put them in this list :
ILOCS = []


# build a model and its trainer

model = FreqNet(
    data_object=db.fft.get(db.metadata.iloc[ILOCS]) if any(ILOCS) else db.fft,
    splits=[.85, .15],
    model_dim=1024,
    kernel_size=3,
    # because we have a lot of layers, we try to speed things up :
    groups=2,
    with_skip_conv=False,
    with_residual_conv=False,
    accum_outputs=0,
    concat_outputs=0,
    pad_input=0,
    **params[PARAMS_INDEX]
     )

trainer = get_trainer(model=model,
                     max_epochs=100,
                      
                     # add a mid-training checkpoint :
                     epochs=[49, 99],
                      
                     # when groups=2, precision=32 is faster than precision=16...
                     precision=32,
                     
                      root_dir=path_to_model,
                     neptune_connector=nep_con,
                     )

# log some infos to the neptune experiment :

nept_exp = [exp for exp in trainer.logger.experiment if exp.__class__.__name__ == "Experiment"]
if any(nept_exp):
    nept_exp[0].append_tags("INDEX=" + str(PARAMS_INDEX),
                         "precision=32",
                         "FreqNet",
                         "db="+db_name,
                         "db_size=" + str(model.data.shape[0]),
                         "receptive_field=" + str(model.receptive_field()),
                         "total_n_layers=" + str(sum(params[PARAMS_INDEX]["n_layers"])))
    
    nept_exp[0].log_text("DB", db_name)
    nept_exp[0].log_text("INDEX", str(PARAMS_INDEX))
    nept_exp[0].log_text("RF", str(model.receptive_field()))
    nept_exp[0].log_text("N_LAYERS", str(sum(params[PARAMS_INDEX]["n_layers"])))
    
    nept_exp[0].log_text("db-name", db_name)
    nept_exp[0].log_text("db-id", nep_con.setup["db"])
    nept_exp[0].log_text("db-iloc", str(ILOCS) if any(ILOCS) else "")
    
    

# Train :

trainer.fit(model)

# upload checkpoints and logs

nep_con.upload_model("model", model, artifacts=("states", "logs"))

### try some prompts

In [ ]:
from random import randint

n_prompts = 2
prompts_indices = [randint(0, model.data.shape[0]-64) for _ in range(n_prompts)]

for n, i in enumerate(prompts_indices):
    output = model.generate(model.data[i:i+64],
                            n_steps=2048)
    
    print("prompt", n, "index", i)
    
    audio(output.squeeze().numpy())


### log a prompt you like

In [ ]:
i = 1234

model.log_audio("prompt=" + str(i), model.generate(model.data[i:i+64], n_steps=2048))

### Download an already trained model

In [ ]:
nep_con.setup["trained"] = "model-upload-test/MOD-134"

nep_con.download_experiment("trained", destination=path_to_model, artifacts="states/")

### Reload a checkpoint

In [ ]:
# replace the "X" in the next line with the (saved) epoch you want to load

path_to_ckpt = path_to_model + "MOD-134/states/epoch=X.ckpt"

model = FreqNet.load_from_checkpoint(path_to_ckpt, data_object=db.fft)